In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [ ]:
E  = 1e4
nu = 0.3
t  = 1e-2#1e-4#1e-2
q  = 1e-1#1e-8#1e-2
D  = (E*t**3)/(12*(1-nu**2))


geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.1))
Draw(mesh)

In [ ]:
def MaterialInv(mat):
    return 1/D*(1/(1-nu)*mat-nu/(1-nu**2)*Trace(mat)*Id(2))

In [ ]:
V = HDivDiv(mesh, order=1)
Q = H1(mesh, order=2, dirichlet=".*")
w_1 = NumberSpace(mesh)
fesm = V*Q*w_1

In [ ]:
sigma, w, w_1= fesm.TrialFunction()
tau, v, v_1 = fesm.TestFunction()
time = 0.0
dt = 0.01

In [ ]:
n = specialcf.normal(2)

def tang(u): return u-(u*n)*n

a = BilinearForm(fesm, symmetric=False)
a += (InnerProduct (MaterialInv(sigma), tau) + div(sigma)*grad(v) \
          + div(tau)*grad(w) )*dx -w_1*v_1*dx\
          + (-(sigma*n) * tang(grad(v)) - (tau*n)*tang(grad(w)))*dx(element_boundary=True)
a.Assemble()

m = BilinearForm(fesm, symmetric=False)
m += w_1*v*dx - w_1*v_1*dx
m.Assemble()

In [ ]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt/4 * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fesm.FreeDofs())

In [ ]:
gfu = GridFunction(fesm)

In [ ]:
t = Parameter(0.0)

In [ ]:
# gausspt = pi**2*sin(pi*(x+1))**4*sin(pi*(y+1))**4*cos(2*pi*t)*(-4+56*pi**2)\
#     + pi**4*sin(pi*(x+1))**2*sin(pi*(y+1))**2*cos(pi*(y+1))**2*cos(2*pi*t)\
#       *(-48*sin(pi*(x+1))**2 + 144*cos(pi*(x+1))**2) \
# + pi**4*sin(pi*(x+1))**2*sin(pi*(y+1))**3*cos(pi*(x+1))*cos(2*pi*t)\
# *(-160*sin(pi*(x+1))*cos(pi*(y+1)) - 240*sin(pi*(y+1))*cos(pi*(x+1)))\
# +pi**4*sin(pi*(y+1))**3*cos(pi*(x+1))**3*cos(2*pi*t)*(96*sin(pi*(x+1))\
# *cos(pi*(y+1))+24*sin(pi*(y+1))*cos(pi*(x+1)))
gausspt = sin(pi*(x+1))**4*sin(pi*(y+1))**4*cos(2*pi*t)
ft = LinearForm(fesm)
ft += gausspt*v*dx
time = 0.0
t.Set(0.0)
gfu.components[1].Set((sin(pi*(x+1))**4*sin(pi*(y+1))**4))
scene = Draw(gfu.components[1],mesh)

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate)
    ft.Assemble()
    k_1 = invmstar*(ft.vec - a.mat*gfu.vec)
    k_2 = invmstar*(ft.vec-a.mat*(gfu.vec + dt/2*k_1))
    gfu.vec.data = gfu.vec + dt/2*(k_1 +k_2)
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print("")
time+=t_intermediate

In [ ]:
# ft = LinearForm(fesm)
# ft += v*dx
# time = 0.0
# t.Set(0.0)
# gfw.Set((sin(pi*(x+1))**4*sin(pi*(y+1))**4))
# scene = Draw(gfw,mesh)

# from ngsolve.internal import *
# visoptions.mminval = 0.0
# visoptions.mmaxval = 0.2
# visoptions.deformation = 0
# visoptions.autoscale = 0

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfsol.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate)
    ft.Assemble()
    k_1 = invmstar*(ft.vec - a.mat*gfw.vec)
    k_2 = invmstar*(ft.vec-a.mat*(gfw.vec + dt/2*k_1))
    gfsol.data = gfw.vec + dt/2*(k_1 +k_2)
    t_intermediate += dt
    scene.Redraw()
print("")
time+=t_intermediate